# Python Syntax Efficiency

In one of my projects, [aiomsg](https://github.com/cjrh/aiomsg), my `setup.py` file contains a section for installing extras:

```python3
extras_require = {
    "dev": ["check-manifest", "colorama", "pygments", "twine", "wheel", "aiorun"],
    "test": ["pytest", "pytest-cov", "portpicker", "pytest-benchmark"],
    "doc": ["sphinx"],
}
```

which gets used a bit further down in the `setup()` function:

```python3
setup(
    name="aiomsg",
    version=open("VERSION").readline().strip(),
    description="Socket-based abstraction for messaging patterns",
    long_description=long_description,
    ...
    packages=find_packages(exclude=["contrib", "docs", "tests"]),
    extras_require=extras_require,   # <== HERE
)

```

This is a pretty common section in many Python projects.  When installing the project, you might do something like this:

```bash
$ pip install aiomsg[dev,test]
```
to install the extras of your choice.

For convenience, I also wanted to add an "all" section to the extras that would install all of them. It should be pretty easy to construct that `"all"` entry, right?

My first approach, which also worked the first time I ran it, was to use a comprehension:

```python
extras_require["all"] = list(
    {pkg for k, v in extras_require.items() for pkg in v}
)
```

Note how we're using a set comprehension to reject duplicates. This code is simple, obvious, very easy to read.

Over time, whenever I saw this "all" line, I began wondering whether there was a better way. This wasn't any kind of formal project; just a scratch at the back of mind that I became curious about. 

Since this involves processing of lists and dictionaries, I decided to experiment with some functional tools.  For example, imagine we invoke `reduce` to build up the "all" entries:

In [16]:
d = dict(a=[1,2,3], b=[2,3,4], c=[3,4,5])
print(d)
from functools import reduce
reduce(
    lambda nd, k: nd['all'].update(d[k]) or nd, 
    d, 
    dict(all=set())
)

{'a': [1, 2, 3], 'b': [2, 3, 4], 'c': [3, 4, 5]}


{'all': {1, 2, 3, 4, 5}}

Ok, that seems to work but it's very clunky.  We don't need to mutate an actual dictionary in the reduction, just returning a list is fine, and then assigning that to the extras entry, exactly as we currently do for the list comprehension:

In [8]:
er = dict()
er['all'] = list(
    reduce(lambda s, k: s.update(d[k]) or s, d, set())
)
er

{'all': [1, 2, 3, 4, 5]}

The callable in the reduction is not great. We're using `s.update(...) or s` because the update method doesn't return itself, which is what `reduce()` requires to work. Well, instead of updating the same set in place, we could use the `union` method which returns a new set on every call. It seems like that would be less efficient, but performance is never going to matter for this use-case so we won't worry about it.

In [11]:
er = dict()
er['all'] = list(
    reduce(lambda s, k: s.union(d[k]), d, set())
)
er

{'all': [1, 2, 3, 4, 5]}

It was at this point that I started to realise that we care only about the values of the existing dictionary here, not the keys; and we can access them immediately with the `.values()` method.  So let's have a look at how that works:

In [12]:
er = dict()
er['all'] = list(
    reduce(lambda s, lst: s.union(lst), d.values(), set())
)
er

{'all': [1, 2, 3, 4, 5]}

It was at this point that I started to doubt my intuition about which syntax choices were going to be best. How had I not seen the `d.values()` much earlier? So then I started to think about how I could use the other basic Python syntax features, like unpacking. Do you see how the reduction above is doing nothing more than feeding each item in `d.values()` into the set `.union()` method? Well `union()` supports unpacking, so that whole complicated, difficult-to-read `reduce()` call completely disappears:

In [13]:
er = dict()
er['all'] = list(set().union(*d.values()))
er

{'all': [1, 2, 3, 4, 5]}

This is a huge improvement over not only the complicated `reduce()` call, but also over my initial set comprehension.

Could we simplify it further? For example, is there a way to get rid of the `union()` call and just use the set constructor directly? There is, but we need `itertools.chain()` to flatten all the sub-sequences that `d.values()` emits:

In [14]:
from itertools import chain

er = dict()
er['all'] = list(set(chain(*d.values())))
er

{'all': [1, 2, 3, 4, 5]}

This code is the simplest, and the shortest, and very likely the most efficient, although I'm not going to benchmark it because I don't care about efficiency for this use-case specifically.

My goal in writing this post is to ask myself why I didn't come up with this solution the first time, when I first added the "all" entry to `extras_require`. I don't have a good answer for that. I certainly don't lack for experience with Python. I suspect it's because my mental _default_ choice has become to start with a list/dict/set comprehension, and then if that gets too complicated, bubble over to an explicit for/while loop.  

Some people will say, "If you were more familiar with functional programming, you would have gotten the best solution first" but I'm not sure that's true. I don't know that it isn't, but I'm wary. In the example described here, we tried a few ways of using the `reduce()` function, and every version of that was strictly worse than the initial comprehension. In our final solution, we take advantage of Python's _iterable unpacking_ syntax, which makes a big difference. It is true that we use the `chain()` function from `itertools`, but the prior version which used only unpacking `d.values()` into the `set.union()` method was also very simple and easy to read.

Moving forward, I'm going to bump up _iterable unpacking_ into _function calls_ much higher in my mental list of approaches when designing new software, and make use of `chain()` for dealing with nested structures.

I'll check back in five years from now and let you know how it went 😉